In [1]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
#tf.enable_eager_execution()

Using TensorFlow backend.


In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax', bias_regularizer=tf.keras.regularizers.l2(0.01))
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       590080    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).repeat(2).batch(64) 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [8]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_accuracy',
    save_best_only=True)

In [9]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [10]:
history = model.fit(train_data,
                    epochs=30,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 14:25:31.738958: {'loss': 2.252607939815521, 'accuracy': 0.441875, 'val_loss': 2.1165035549242783, 'val_accuracy': 0.5149}
Epoch 1 ends at 14:27:36.342308: {'loss': 1.338231166601181, 'accuracy': 0.6125125, 'val_loss': 1.227925189741098, 'val_accuracy': 0.5846}
Epoch 2 ends at 14:29:42.138436: {'loss': 0.9187158030748367, 'accuracy': 0.7023375, 'val_loss': 0.9297789247931948, 'val_accuracy': 0.6977}
Epoch 3 ends at 14:31:48.459174: {'loss': 0.6774426629781723, 'accuracy': 0.7713125, 'val_loss': 0.7447370152184918, 'val_accuracy': 0.7462}
Epoch 4 ends at 14:33:56.617363: {'loss': 0.5298946601271629, 'accuracy': 0.818275, 'val_loss': 0.9709497807891505, 'val_accuracy': 0.6876}
Epoch 5 ends at 14:36:01.809833: {'loss': 0.40691206282377246, 'accuracy': 0.85755, 'val_loss': 0.801623605618811, 'val_accuracy': 0.7405}
Epoch 6 ends at 14:38:06.600600: {'loss': 0.31278612371683123, 'accuracy': 0.889675, 'val_loss': 0.7063474468175014, 'val_accuracy': 0.7869}
Epoch 7 ends at 14:4

In [11]:
model.evaluate(test_data)

157/157 [==============================].6005 - accuracy: 0.85 - 0s 68ms/step - loss: 0.8668 - accuracy: 0.82 - 0s 61ms/step - loss: 0.8802 - accuracy: 0.83 - 0s 59ms/step - loss: 0.8299 - accuracy: 0.82 - 0s 58ms/step - loss: 0.8273 - accuracy: 0.82 - 0s 56ms/step - loss: 0.7870 - accuracy: 0.82 - 0s 55ms/step - loss: 0.8308 - accuracy: 0.81 - 0s 54ms/step - loss: 0.8693 - accuracy: 0.80 - 0s 53ms/step - loss: 0.8250 - accuracy: 0.81 - 1s 53ms/step - loss: 0.8676 - accuracy: 0.81 - 1s 53ms/step - loss: 0.8423 - accuracy: 0.81 - 1s 52ms/step - loss: 0.8678 - accuracy: 0.81 - 1s 51ms/step - loss: 0.8725 - accuracy: 0.81 - 1s 51ms/step - loss: 0.8657 - accuracy: 0.81 - 1s 50ms/step - loss: 0.8682 - accuracy: 0.81 - 1s 50ms/step - loss: 0.9109 - accuracy: 0.80 - 1s 51ms/step - loss: 0.9057 - accuracy: 0.80 - 1s 52ms/step - loss: 0.9130 - accuracy: 0.80 - 1s 51ms/step - loss: 0.9137 - accuracy: 0.80 - 1s 51ms/step - loss: 0.9335 - accuracy: 0.80 - 1s 51ms/step - loss: 0.9190 - accuracy: 0.

[1.0005776661976127, 0.8053]

In [12]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))

In [ ]:
#repeat 숫자를 4->2로 낮춤. epoch 속도 증가, epoch 처음 올라갈때 정확도 떨어짐. (l2 추가하니까, 속도가 조금 느려진 것 같기도 함.)